# Training an Object Detection model using Detectron2

This 5-minute quickstart tutorial demonstrates how to train Detectron2 on object detection datasets. In this dataset each example contains a bounding box and a class label surrounding a physical object within an image scene. Using this labeled data, we train a model to predict classes of objects in an image and their physical locations.

This notebook demonstrates how to train a state-of-the-art object detection model using Detectron2 and use it to produce `pred_probs`, which will help detect label errors in Object detection datasets. 

To identify label errors, we provide an [example](https://github.com/cleanlab/examples/) notebook on our GitHub repository on [Finding Label Errors in Object Detection Datasets](https://github.com/cleanlab/cleanlab/blob/master/docs/source/tutorials/object_detection.ipynb). Here we fit a state-of-the-art neural network initialized from a pretrained [X-101](https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md#imagenet-pretrained-models) network backbone.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cleanlab/examples/blob/master/object_detection/detectron2_training.ipynb)

In [1]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import pickle
# import some common libraries
import numpy as np
import os, json, cv2, random
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances


In [2]:
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/DATASET_annotations/instances_val2017_5labels.json"
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/DATASET_annotations/instances_train2017_5labels.json"
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/tutorial_obj/dataset.pkl"

File ‘instances_val2017_5labels.json’ already there; not retrieving.

File ‘instances_train2017_5labels.json’ already there; not retrieving.

File ‘dataset.pkl’ already there; not retrieving.



Before you begin training on a custom dataset, be sure to review the COCO dataset guidelines for formatting your data, which can be found on their [website](https://cocodataset.org/#format-data).

To use a custom dataset named "my_dataset" with detectron2, users must implement a function that returns the items in the dataset and then inform detectron2 about this function. For instance, a subset of the labels ["car", "chair", "cup", "person", and "traffic light"] are used for training and detecting errors in this notebook.

In [3]:
IMAGE_PATH = "../data/coco/"
TRAIN_PATH = os.path.join(IMAGE_PATH,"train2017")
VAL_PATH = os.path.join(IMAGE_PATH,"val2017")
register_coco_instances("my_dataset_train", {}, "instances_train2017_5labels.json",
                        TRAIN_PATH)
register_coco_instances("my_dataset_val", {}, "instances_val2017_5labels.json",
                        VAL_PATH)


We define the configuration settings for training an object detection model using Detectron2. The model architecture used in this example is "faster_rcnn_X_101_32x8d_FPN_3x" from the COCO-Detection model zoo. The training data is specified by the "my_dataset_train" dataset and validation data is specified by the "my_dataset_val" dataset which refer to COCO2017 train and val containing only the subset of labels specified before.

The number of worker threads is set to 2 and the batch size is set to 2.
The learning rate and maximum number of iterations are also specified. The model is initialized from the COCO-Detection model zoo and the output directory for the trained model is created. Finally, the configuration is passed to the DefaultTrainer class for training the object detection model.

<strong>Note:</strong> The number of iterations was set based on [early stopping.](https://en.wikipedia.org/wiki/Early_stopping#:~:text=In%20machine%20learning%2C%20early%20stopping,training%20data%20with%20each%20iteration.)

In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 30000    # 
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # only 5 classes ["car", "chair", "cup", "person", and "traffic light"] 
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)



## Train the model


In [ ]:
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[05/03 17:42:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (6, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (6,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (20, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (20,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[05/03 17:42:15 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/03 17:42:22 d2.utils.events]:  eta: 2:16:58  iter: 19  total_loss: 2.598  loss_cls: 1.799  loss_box_reg: 0.7445  loss_rpn_cls: 0.01884  loss_rpn_loc: 0.02223    time: 0.2765  last_time: 0.3058  data_time: 0.0108  last_data_time: 0.0033   lr: 4.9953e-06  max_mem: 5256M
[05/03 17:42:28 d2.utils.events]:  eta: 2:14:34  iter: 39  total_loss: 2.561  loss_cls: 1.739  loss_box_reg: 0.7628  loss_rpn_cls: 0.01208  loss_rpn_loc: 0.008133    time: 0.2755  last_time: 0.2702  data_time: 0.0027  last_data_time: 0.0026   lr: 9.9902e-06  max_mem: 5427M
[05/03 17:42:33 d2.utils.events]:  eta: 2:14:46  iter: 59  total_loss: 2.389  loss_cls: 1.594  loss_box_reg: 0.7617  loss_rpn_cls: 0.01646  loss_rpn_loc: 0.01395    time: 0.2760  last_time: 0.3034  data_time: 0.0027  last_data_time: 0.0019   lr: 1.4985e-05  max_mem: 5546M
[05/03 17:42:38 d2.utils.events]:  eta: 2:14:02  iter: 79  total_loss: 2.279  loss_cls: 1.406  loss_box_reg: 0.8342  loss_rpn_cls: 0.007432  loss_rpn_loc: 0.01523    time: 0.2735  

## Inference & evaluation using the trained model
If you wish to load a trained model to run inference

In [ ]:
evaluator = COCOEvaluator("my_dataset_val", output_dir="../output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

The given code block implements a function "conv" that converts the output of Detectron2 to a format that can be used by Cleanlab for identifying label errors. The function accepts the predicted instances and the number of classes as inputs. It processes the predicted bounding boxes and scores for each instance, and outputs a list of numpy arrays containing the bounding boxes and scores for each class.

In [ ]:
def conv(ins,num_classes):
    fields = ins.get_fields()
    boxes = fields['pred_boxes'].tensor.numpy()
    res = [[] for i in range(num_classes)]
    for i in range(0,len(fields['pred_classes'])):
        pred_class = fields['pred_classes'][i].item()
        probs = ins.get_fields()['scores'][i].item()
        box_cord = list(boxes[i])
        box_cord.append(probs)
        res[pred_class].append(box_cord)
    res2 = []
    for i in res:
        if len(i)==0:
            q = np.array(i,dtype=np.float32).reshape((0,num_classes))
        else:
            q = np.array(i,dtype=np.float32)
        res2.append(q)
    return res2
        


To perform inference and testing on the tutorial notebook linked here, we utilize a limited portion of the validation set of COCO 2017. To find label errors on this subset, please run our tutorial notebook on [Finding Label Errors in Object Detection Datasets](https://github.com/cleanlab/cleanlab/blob/master/docs/source/tutorials/object_detection.ipynb) to find label errors.

In [ ]:
dataset = pickle.load(open("dataset.pkl",'rb'))
results = []
for i in dataset:
    im_name = os.path.join(VAL_PATH, i['seg_map'].replace(".png",'.jpg'))
    im = cv2.imread(im_name)
    outputs = predictor(im)
    results.append(conv(outputs["instances"].to("cpu"),cfg.MODEL.ROI_HEADS.NUM_CLASSES))
    
pickle.dump(results,open("results.pkl",'wb'))